# Install/import packages

In [ ]:
# install necessary packages
!pip install pymatgen
!pip install mp_api

In [ ]:
# import necessary packages
import pymatgen
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.core.periodic_table import Element
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from mp_api.client import MPRester
from pymatgen.core import Structure
import numpy as np
import pandas as pd

# Define API key and structural descriptors

In [ ]:
# Define API key for data extraction from the MPD
API_KEY = "INSERT YOUR API KEY HERE"

In [ ]:
'''
Compute 5 structural descriptors including the following:
1. Lithium-Lithium Bonding (LLB)
2. Sublattice Bond Ionicity (SBI)
3. Anion Framework Coordination (AFC)
4. Li-Anion Separation Distance (LASD)
5. Li-Li Separation Distance (LLSD)
'''

def compute_descriptors(structure):

    results = {}

    # Initialize neighbor analysis tool
    cnn = CrystalNN()

    # Get Li sites
    li_sites = [site for site in structure if site.species_string == "Li"]

    # 1. Li-Li Bond Count (LLB)
    li_bonds = 0
    for li in li_sites:
        neighbors = cnn.get_nn_info(structure, structure.index(li))
        li_neighbors = [n for n in neighbors if n["site"].species_string == "Li"]
        li_bonds += len(li_neighbors)
    results["LLB"] = li_bonds / len(li_sites) if li_sites else 0

    # 2. Sublattice Bond Ionicity (SBI)
    ionicity_sum = 0
    for li in li_sites:
        neighbors = cnn.get_nn_info(structure, structure.index(li))
        for neighbor in neighbors:
            delta_X = abs(Element("Li").X - Element(neighbor["site"].species_string).X)
            ionicity_sum += delta_X
    results["SBI"] = ionicity_sum / len(structure) if len(structure) > 0 else 0

    # 3. Anion Framework Coordination (AFC)
    anions = [site for site in structure]
    total_coord = sum(len(cnn.get_nn_info(structure, structure.index(anion))) for anion in anions)
    results["AFC"] = total_coord / len(anions) if anions else 0

    # 4. Li-Anion Separation Distance (LASD)
    anion_sites = [site for site in structure]
    if li_sites and anion_sites:
        distances = [li.distance(anion) for li in li_sites for anion in anion_sites]
        results["LASD"] = np.mean(distances)
    else:
        results["LASD"] = 0

    # 5. Li-Li Separation Distance (LLSD)
    if len(li_sites) >= 2:
        distances = []
        for i in range(len(li_sites)):
            for j in range(i+1, len(li_sites)):
                distances.append(li_sites[i].distance(li_sites[j]))
        results["LLSD"] = np.mean(distances)
    else:
        results["LLSD"] = 0

    return results

# Create training set

Training data is based off of Li-based compounds in the MPD with DFT-verified ionic conductivity values according to ables 1 and 2 in the paper "Machine Learning-Assisted Discovery of Solid Li-Ion Conducting Materials" by Sendek et. al.

Link: https://pubs.acs.org/doi/10.1021/acs.chemmater.8b03272

In [ ]:
# MPID values manually extracted from tables 1 and 2
mpid_list = [
    # random (table 2)
    "mp-6674", "mp-8609", "mp-28567", "mp-12160",
    "mp-6787", "mp-560072", "mp-7941", "mp-13772",
    "mp-8452", "mp-770932", "mp-12829", "mp-27811",
    "mp-8180", "mp-7971", "mp-8449", "mp-558045", "mp-562394",
    "mp-510073", "mp-7744", "mp-989562", "mp-8892",
    # machine learning guided (table 1)
    "mp-554076", "mp-532413", "mp-558219",
    "mp-29410", "mp-676361", "mp-643069", "mp-34477",
    "mp-676109", "mp-559238", "mp-866665", "mp-8751", "mp-15789",
    "mp-15790", "mp-15791", "mp-561095", "mp-8430", "mp-22905"
]

I did not query these values. I manually copied and pasted values from the full excel spreadsheet. So we can either add the code in to extract based on MPID or just upload the excel spreadsheets as is (which I understand might be feel incomplete to do)

# Retrieve parameters of all Li-based compounds from the Materials Project Database (MPD) for full analysis of potential new superionic materials

In [ ]:
'''
Retrieve compound properties from the MPD to calculate 5 structural
descriptors including:
1. MPD ID
2. Chemical formula
3. Number of elements in compound
4. Band gap in eV
5. Thermodynamic stability in eV
6. Gibbs free energy in eV
'''

# Query Materials Project database for all Li-based compounds
with MPRester(API_KEY) as mpr:
    query = mpr.materials.summary.search(
        elements=["Li"],
        fields=["material_id", "structure", "nelements", "band_gap",
                "energy_above_hull", "formation_energy_per_atom"]
    )

In [ ]:
# Process all materials and compute descriptors
data = []
for doc in tqdm(query, total=len(query)):
    try:
        struct = doc.structure
        descriptors = compute_descriptors(struct)
        descriptors.update({
            "material_id": doc.material_id,
            "formula": struct.formula,
            "num_elements": doc.nelements,
            "bandgap": doc.band_gap,
            "energy_above_hull": doc.energy_above_hull,
            "formation_energy_per_atom": doc.formation_energy_per_atom
        })
        data.append(descriptors)
    except Exception as e:
        print(f"Error processing {doc.material_id}: {str(e)}")

In [ ]:
# Create DataFrame and save to .csv file
Li_materials_df = pd.DataFrame(data)
#csv = df.to_csv("li_structural_descriptors.csv", index=False)

print(f"Successfully processed {len(data)}/{len(query)} materials")
print(df.head())

**Pre-filtering MPD Materials**

Many of the 20,000+ Li-based materials in the Materials Project Database can be pre-filtered based on the following parameters:

1. **Band gap**: Materials with a band gap lower than 1 eV can conduct electrons too easily and thus would not be effective ion conductors.
2. **Energy above hull**: Materials need to be thermodynamically stable to operate as an effective ion conductor, so $E_{hull}$ should equal 0 eV.
3. **Contains transition metals**: Previous studies have shown that Li-based materials without transition metals typically do not operate as effective ion conductors.

This filtering process decreases the size of our dataset from 21,756 to 452 Li-based materials.

In [ ]:
# define list of transition metals
transition_metals = [
    "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",  # 3rd period
    "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",  # 4th period
    "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",        # 5th period
    "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds", "Rg", "Cn"        # 6th period
]

# filter dataframe of Li-based materials based on parameters listed above
for material_index in range(len(Li_materials_df)):
  if (Li_materials_df.bandgap[material_index] < 1) or \
     (not Li_materials_df.energy_above_hull[material_index] == 0) or \
     (not any(metal in Li_materials_df.formula[material_index] for metal in
         transition_metals)):
      Li_materials_df = Li_materials_df.drop(index=material_index)

In [ ]:
# Export values to .csv file
csv = Li_materials_df.to_csv("li_structural_descriptors_filtered.csv", index=False)
from google.colab import files
files.download('li_structural_descriptors_filtered.csv')